In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL 설정
base_url = "http://llumar.co.kr/sub_bbs/bbs_list.php?curr_page={}&bcode=11&list_count=1&sch_option=&sch_str="

# 데이터 저장용 리스트
data = []

# 페이지 순회
page_num = 1
while True:
    url = base_url.format(page_num)
    response = requests.get(url)
    response.encoding = 'euc-kr'
    soup = BeautifulSoup(response.text, 'html.parser')

    # 테이블 행 추출
    rows = soup.select('table > tbody > tr')
    valid_data_found = False

    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 4:
            no = cols[0].text.strip()
            title_tag = cols[1].find('a')
            title = title_tag.text.strip()
            link = title_tag['href']
            date = cols[2].text.strip()
            views = cols[3].text.strip()

            # 본문 내용 가져오기
            full_url = f"http://www.llumar.co.kr/sub_bbs/{link}"
            detail_response = requests.get(full_url)
            detail_response.encoding = 'euc-kr'
            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
            content_div = detail_soup.find('div', class_='bbs_view')
            if content_div:
                content = ' '.join(content_div.stripped_strings)
            else:
                content = "No content found"

            data.append([no, title, date, views, content])
            valid_data_found = True

    if not valid_data_found:
        break

    next_page_num = page_num + 1
    next_page_link = soup.find('a', href=lambda href: href and f'curr_page={next_page_num}' in href)
    if not next_page_link:
        break

    page_num += 1


Data saved to promotion_scraping_results.csv
       No                                                 제목         작성일  \
0  NOTICE  뉴스2데이 [인물탐구] 루마썬팅 김우화 대표 - 수많은 최초신화로 혁신적 마케팅의 ...  2018.09.25   
1  NOTICE      뉴스2데이 [인물탐구] 루마틴팅 김우화 대표 - 썬팅업계를 뒤집은 역발상의 승부사  2018.09.23   
2  NOTICE             [인터뷰기사] 문화일보 골프와 나에 소개된 루마코리아 김우화 대표이사  2018.01.23   
3      67                       루마 CM송 2편 - 대한민국 3대 마요~ 루마요~  2021.05.27   
4      66                             루마 CM송 1편 - 루마버텍스 SONG  2021.05.27   

     조회                                                 본문  
0  8441  [ 인물탐구 ] 루마썬팅 김우화\r\n대표 ② 성과 : 수많은 ‘ 최초\r\n신화 ...  
1  8354  [인물탐구] 루마윈도틴팅 김우화 대표 ① 경력: 삼성전자 나와 썬팅업계를 뒤집은 역...  
2  8637  문화일보, 스포츠 - 골프와 나에 소개된 루마코리아 김우화 대표이사 ‘ 선팅 神話 ...  
3  1569                                                     
4  1722                                                     


In [2]:

content

'매일경제TV 인터뷰 스틸컷'

In [5]:
print(df.loc[3, '본문'])
